In [121]:
import numpy as np
import sklearn
from sklearn.cluster import KMeans

X_train = np.load("./scene_categorization/train_D.npy", allow_pickle=True)
Y_train = np.load("./scene_categorization/train_gs.npy", allow_pickle=True)
X_train_clusters = np.concatenate([x for x in X_train], axis=0)
X_train_clusters = X_train_clusters
np.random.shuffle(X_train_clusters)
# computing word dictionary on 50,000 shuffled points instead of 413,971, took about 2 minutes
kclusters = KMeans(n_clusters = 100, random_state=0, max_iter = 300).fit(X_train_clusters[:50000])
kclusters.labels_

array([44, 58, 45, ..., 92, 88, 76])

In [122]:
# calculate histogram for each image in training set
print(X_train.shape)
im_hists = []
for im_feats in X_train:
  clusters = kclusters.predict(im_feats)
  hist = np.zeros(shape=(100))
  for centroid in clusters:
    hist[centroid] += 1
  im_hists.append(hist)

(1888,)


In [123]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# train classifier on the training images' histograms with their labels
classifier = KNeighborsClassifier(n_neighbors=6).fit(im_hists, Y_train.ravel())
X_test = np.load("./scene_categorization/test_D.npy", allow_pickle=True)
Y_test = np.load("./scene_categorization/test_gs.npy", allow_pickle=True)

# build histograms
test_im_hists = []
for test_im_feats in X_test:
  test_clusters = kclusters.predict(test_im_feats)
  hist = np.zeros(shape=(100))
  for centroid in test_clusters:
    hist[centroid] += 1
  test_im_hists.append(hist)

#output classification scores and confusion matrix
preds = classifier.predict(test_im_hists)
print("Confusion Matrix:")
print(confusion_matrix(preds, Y_test.ravel()))
print()
print("Classification Report:")
print(classification_report(preds, Y_test.ravel()))

Confusion Matrix:
[[62  0 29  6 12 36  3 16]
 [ 1 92  1  7  6  6  8  4]
 [17  0 50  2  2  4  1  6]
 [ 0  0  1 48  1  1 10 14]
 [ 7  2  3  4 57 12 11  9]
 [12  4  8 11 13 38  7 12]
 [ 1  1  6 13  9  0 55 14]
 [ 0  1  2  9  0  3  5 25]]

Classification Report:
              precision    recall  f1-score   support

           1       0.62      0.38      0.47       164
           2       0.92      0.74      0.82       125
           3       0.50      0.61      0.55        82
           4       0.48      0.64      0.55        75
           5       0.57      0.54      0.56       105
           6       0.38      0.36      0.37       105
           7       0.55      0.56      0.55        99
           8       0.25      0.56      0.34        45

    accuracy                           0.53       800
   macro avg       0.53      0.55      0.53       800
weighted avg       0.57      0.53      0.54       800

